# 任务5：订单调度分析

## 学习目标

- 挖掘巡游车空载率与经纬度的关系；  
- 挖掘巡游车订单运距与经纬度的关系；  
- 挖掘网约车空载率与经纬度的关系；  
- 挖掘网约车订单运距与经纬度的关系；

## 经纬度编码

geohash用一个字符串表示经度和纬度两个坐标。利用geohash，只需在一列上应用索引即可。  
其次，geohash表示的并不是一个点，而是一个矩形区域。比如编码wx4g0ec19，它表示的是一个矩形区域。 使用者可以发布地址编码，既能表明自己位于北海公园附近，又不至于暴露自己的精确坐标，有助于隐私保护。    
   - 热力图展示中，用于geohash点聚合功能，可以提高大量点前端渲染的性能  
   - 地址聚类中，通过geohash计算，进行地址归类  
   - 地址信息缓存时，通过geohash为key，来索引缓存信息

In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [11]:
import geohash

In [1]:
import pandas as pd
import numpy as np
import glob


MAX_ROWS = None # 文件读取行数

taxiorder2019 = pd.read_csv('taxiOrder20190531.csv', nrows=None,
                           dtype = {
                               'GETON_LONGITUDE': np.float32,
                               'GETON_LATITUDE': np.float32,
                               'GETOFF_LONGITUDE': np.float32,
                               'GETOFF_LATITUDE': np.float32,
                               'PASS_MILE': np.float16,
                               'NOPASS_MILE': np.float16,
                               'WAITING_TIME': np.float16
                           })


taxiorder2019['GETON_DATE'] = pd.to_datetime(taxiorder2019['GETON_DATE']) 
taxiorder2019['GETOFF_DATE'] = pd.to_datetime(taxiorder2019['GETOFF_DATE'])

taxiorder2019 = taxiorder2019.rename(columns={'CAR_NO':'CARNO'})
taxiorder2019.sort_values(by=['CARNO','GETON_DATE'], inplace=True)
taxiorder2019.reset_index(inplace=True, drop=True)

In [38]:
# taxiorder2019.apply(lambda x: geohash.encode(x['GETON_LATITUDE'], x['GETON_LONGITUDE'],  precision=5), axis=1)
# 字符串越长，表示的范围越精确

0         wsk52
1         wsk52
2         wsk52
3         ws7gr
4         ws7gr
          ...  
248156    ws7gr
248157    ws7gn
248158    wsk50
248159    ws7gp
248160    ws7gp
Length: 248161, dtype: object

In [43]:
taxiorder2019['geohash'] = taxiorder2019.apply(lambda x: geohash.encode(x['GETON_LATITUDE'], x['GETON_LONGITUDE'], precision=8), axis=1)
taxiorder2019['geohash'].value_counts().head(10)

7zzzzzzz    6335
ws7unv0q     809
wsk584c9     727
ws7unv0r     598
ws7grb9s     590
wsk52k80     454
wsk584cd     410
ws7grbsy     407
ws7grb6d     396
wsk52dj9     355
Name: geohash, dtype: int64

In [23]:
taxiorder2019['geohash'].value_counts().iloc[1:11].index

Index(['ws7unv0q', 'wsk584c9', 'ws7unv0r', 'ws7grb9s', 'wsk52k80', 'wsk584cd',
       'ws7grbsy', 'ws7grb6d', 'wsk52dj9', 'wsk584c3'],
      dtype='object')

其中`7zzzzzzz`为经纬度异常点，可以剔除掉。接下来我们统计下热门经纬度的中心：

In [16]:
for idx in taxiorder2019['geohash'].value_counts().iloc[1:11].index: #看别人怎么索引的，不要第一个，异常点，取了10个
    df = taxiorder2019[taxiorder2019['geohash'] == idx]
    print(idx, df['GETON_LONGITUDE'].mean(), df['GETON_LATITUDE'].mean())

ws7unv0q 118.07034301757812 24.637990951538086
wsk584c9 118.12725830078125 24.536975860595703
ws7unv0r 118.07035064697266 24.637996673583984
ws7grb9s 118.1165542602539 24.481197357177734
wsk52k80 118.13651275634766 24.502304077148438
wsk584cd 118.12763214111328 24.537063598632812
ws7grbsy 118.12097930908203 24.481435775756836
ws7grb6d 118.11732482910156 24.479427337646484
wsk52dj9 118.1548080444336 24.488798141479492
wsk584c3 118.12710571289062 24.536853790283203


In [50]:
taxiorder2019['geohash'].values

array(['wsk52tjv', 'wsk52dj9', 'wsk52tjy', ..., 'wsk502er', 'ws7gp0tu',
       'ws7gp436'], dtype=object)

In [57]:
geohash.decode('wsk52tjv')

('24.506', '118.155')

In [49]:
geohash.decode(taxiorder2019['geohash'].values)

KeyError: 'wsk52tjv'

## 订单调度分析

巡游车根据等待时间统计具体位置：

In [26]:
legal_hash

Index(['7zzzzzzz', 'ws7unv0q', 'wsk584c9', 'ws7unv0r', 'ws7grb9s', 'wsk52k80',
       'wsk584cd', 'ws7grbsy', 'ws7grb6d', 'wsk52dj9',
       ...
       'ws7grb93', 'ws7gxspe', 'wsk528bu', 'ws7gpxhj', 'ws7gqc1d', 'ws7gr9zj',
       'ws7gph8s', 'ws7gry6b', 'ws7gpmew', 'wsk589md'],
      dtype='object', length=5000)

In [25]:
legal_hash = taxiorder2019['geohash'].value_counts().iloc[:5000].index
taxiorder2019[taxiorder2019['geohash'].isin(legal_hash)].groupby(['geohash'])['WAITING_TIME'].mean().sort_values(ascending=False) #这个isin()就很灵性了

geohash
7zzzzzzz           inf
wsk528nu    5388.00000
ws7grh36    4568.00000
ws7gpxvp    1576.00000
ws7gnsy0    1433.00000
               ...    
wsk52h6y      66.00000
ws7gx9q2      65.50000
wsk53nj1      63.53125
ws7gx9nh      57.62500
ws7grb2e      48.50000
Name: WAITING_TIME, Length: 5000, dtype: float16

网约车根据等待时间统计具体位置：

In [32]:
wycorder2019=pd.read_csv('wycOrder20190531.csv')

In [34]:
wycorder2019['DEP_TIME'] = pd.to_datetime(wycorder2019['DEP_TIME'])
wycorder2019['BOOK_DEP_TIME'] = pd.to_datetime(wycorder2019['BOOK_DEP_TIME'])

In [36]:
wycorder2019['WAITING_TIME'] = (wycorder2019['DEP_TIME'] - wycorder2019['BOOK_DEP_TIME']).dt.seconds
wycorder2019['geohash'] = wycorder2019.apply(lambda x: geohash.encode(x['DEP_LATITUDE'], x['DEP_LONGITUDE'], precision=8), axis=1)
wycorder2019[wycorder2019['geohash'].isin(legal_hash)].groupby(['geohash'])['WAITING_TIME'].mean().sort_values(ascending=False)

geohash
wsk533xq    86399.0
ws7gpwk6    86399.0
ws7grvxz    86399.0
ws7gnuhs    86399.0
wsk50zqt    86399.0
             ...   
ws7grgqh        0.0
ws7grgq6        0.0
ws7grgq5        0.0
ws7grgq1        0.0
ws7grbke        0.0
Name: WAITING_TIME, Length: 872, dtype: float64

通过统计发现，网约车与巡游车打不到车的位置区域存在部分重叠，但还是会有交叉的情况

- 比如打不到网约车，也打不到出租车；
- 比如打不到网约车，但能打到出租车；


## 课堂任务

1. 有没有其他对地理位置做聚合的方法？  
2. 如何定量衡量一个区域打不到车的情况？

【回答】1. 用经纬度合成一个字符串字段进行聚合  
&emsp;&emsp;&emsp;&emsp;2. 等待时间特别长的位置

## 打卡任务

1. 使用geohash发掘2019年端午假期，网约车热门的下车地点（Top3）；  
2. 使用geohash发掘2019年端午假期，网约车热门的打车（上车）地点（Top3）；  
3. 使用geohash分别发掘2019和2020年端午假期，网约车打车预计上车时间到上车时间时间最长且地点出现次数大于100的地点（两年的Top1）。